# Adjusted weights Cox PH model, penalized

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

import pickle

from weighted_concordance import *

In [3]:
# pd.set_option('display.max_columns', None,'display.max_rows',20)

In [4]:
from lifelines import CoxPHFitter
from lifelines.utils import datetimes_to_durations, k_fold_cross_validation, concordance_index

In [5]:
# importing data
X_tr = pd.read_csv("D:/compiled_data/X_tr.csv").set_index("csid")

y = pd.read_csv("D:/compiled_data/y.csv").set_index("csid")

Converting date-times from string format to datetime format.

In [8]:
def date_time_conversion(date):
    return(datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ"))

In [9]:
# Coverting the times into datetime format
y_tr = y.assign(
    event = y.ep_CKB0020_combined_ep,
    start_time = y.study_date.map(date_time_conversion),
    event_time = y.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y.columns)
y_tr["duration"] = datetimes_to_durations(y_tr["start_time"],y_tr["event_time"])[0]
y_tr["subcohort"] = y["subcohort"]
y_tr = y_tr.drop(columns = ["start_time","event_time"])
y_tr

,event,duration,subcohort
csid,,,
2022-00229-1799093,1,1192.0,False
2022-00229-4744170,1,1690.0,False
2022-00229-1082517,1,2035.0,False
2022-00229-3397875,1,2530.0,False
2022-00229-4372175,1,2216.0,False
...,...,...,...
2022-00229-5565114,0,4495.0,True
2022-00229-4902412,0,3448.0,True
2022-00229-4823882,0,895.0,True


## Fitting Model

Unlike unregularised, dimension does not seem to have an issue. Now we need to select the regulariser using cross-validation.

See "Unweighted Cox PH with PCA" file for more detailed code annotation. Errors are suppressed as usually there is a successful analysis on at least one of the folds, so we ignore the failures.

In [10]:
from sklearn.decomposition import PCA
from lifelines.utils import k_fold_cross_validation, concordance_index
from sklearn.model_selection import KFold

In [11]:
cox_adjusted_pca_scores =  pd.DataFrame({
    'penalizer' : [],
    'Weight': [],
    'score': []
})
cox_adjusted_pca_scores

,penalizer,Weight,score


In [12]:
alpha = len(y_tr.query("subcohort == True"))/1400

for i in range(10,15):
    for j in range(1,round(1/alpha)*8):
        
        print("Penalty:",i/10, "Weight:", j/4)
    
        kf = KFold(n_splits = 15)
        kf.get_n_splits(X_tr)

        scores = []

        for train_index, test_index in kf.split(X_tr):
            X_tr_train, X_tr_test = X_tr.iloc[train_index], X_tr.iloc[test_index]
            y_tr_train, y_tr_test = y_tr.iloc[train_index], y_tr.iloc[test_index]

            # creating a case dataframe
            cases_barlow_df = X_tr_train.join(y_tr_train).query("subcohort == False")

            cases_barlow_df = cases_barlow_df.assign(
                # setting events outside subcohort to start just before they occur
                start_time = lambda df: df["duration"] - 0.001,
                # adding appropriate weight
                weight = 1,
                subcohort = False
            )

            # creating a subcohort dataframe
            subcohort_barlow_df = X_tr_train.join(y_tr_train).query("subcohort == True")

            # sampling proportion of cohort
            alpha = len(subcohort_barlow_df)/1400

            subcohort_barlow_df = subcohort_barlow_df.assign(
                    # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
                    duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
                    # the events start from the origin
                    start_time = 0, 
                    event = False,
                    weight = j/4,
                    subcohort = True
                )

            case_subcohort_barlow_df = pd.concat([cases_barlow_df,subcohort_barlow_df]).drop(columns = "subcohort")

            cph = CoxPHFitter(penalizer = i/10)
            try:
                cph.fit(case_subcohort_barlow_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

                event_times = y_tr_test["duration"]
                event_observed = y_tr_test["event"]

                test_preds = cph.predict_partial_hazard(X_tr_test)
                scores.append(weighted_concordance(event_times,test_preds, event_observed,0.0011604684001529089))
            except:
                pass
            
        cox_adjusted_pca_scores = cox_adjusted_pca_scores.append({
            'penalizer' : i/10,
            "Weight:": j/4,
            'score': np.mean(scores)
        }, ignore_index = True)


Penalty: 1.0 Weight: 0.25


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

Penalty: 1.0 Weight: 0.5


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

Penalty: 1.0 Weight: 0.75


...

C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

Penalty: 1.4 Weight: 5.5


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

Penalty: 1.4 Weight: 5.75


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

In [13]:
pd.set_option('display.max_columns', None,'display.max_rows', None)

In [17]:
cox_adjusted_pca_scores.sort_values("score").sort_values("score")#.pivot(index = "penalizer", columns = "Weight:", values = "score")

,penalizer,Weight,score,Weight:
51,1.2,NaN,0.671739,3.50
50,1.2,NaN,0.671804,3.25
24,1.1,NaN,0.671807,2.50
22,1.1,NaN,0.671816,2.00
58,1.2,NaN,0.671865,5.25
59,1.2,NaN,0.671865,5.50
60,1.2,NaN,0.671865,5.75
53,1.2,NaN,0.671865,4.00
56,1.2,NaN,0.671865,4.75
57,1.2,NaN,0.671865,5.00


Best combination was 1.0 penaliser 0.75 weight.

In [18]:
# creating a case dataframe
cases_barlow_df = X_tr.join(y_tr).query("subcohort == False")

In [19]:
cases_barlow_df = cases_barlow_df.assign(
    # setting events outside subcohort to start just before they occur
    start_time = lambda df: df["duration"] - 0.001,
    # adding appropriate weight
    weight = 1,
    subcohort = False
)

In [20]:
# creating a subcohort dataframe
subcohort_barlow_df = X_tr.join(y_tr).query("subcohort == True")

In [21]:
# sampling proportion of cohort
subcohort_barlow_df = subcohort_barlow_df.assign(
        # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
        duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
        # the events start from the origin
        start_time = 0, 
        event = False,
        weight = 0.75,
        subcohort = True
    )

In [22]:
case_subcohort_barlow_df = pd.concat([cases_barlow_df,subcohort_barlow_df]).drop(columns = "subcohort")

cph = CoxPHFitter(penalizer = 1)

cph.fit(case_subcohort_barlow_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

<lifelines.CoxPHFitter: fitted with 1041.25 total observations, 446.25 right-censored observations>

In [23]:
pickle.dump(cph, open('penalised_adjusted_cox_wc.pkl', 'wb'))